In [ ]:
import time
import rtsvg
rt = rtsvg.RACETrack()
from xwords import XWords, XWordsSolver
import copy
import os
_dir_    = '../../../data/crossword_puzzle_screenshots/'
_files_  = os.listdir(_dir_)
entries_file    = None
geometries_file = None
blockers_file   = None
answers_file    = None
for _file_ in _files_:
    if   'entries'    in _file_: entries_file    = _dir_ + _file_
    elif 'geometries' in _file_: geometries_file = _dir_ + _file_
    elif 'blockers'   in _file_: blockers_file   = _dir_ + _file_
    elif 'answers'    in _file_: answers_file    = _dir_ + _file_
xwords = XWords(rt, entries_file, geometries_file, blockers_file, answers_file)
results_lu = {}
xwords

In [ ]:
import os
import importlib.util
import inspect
def getClassesFromFile(file_path):
    classes = []
    module_name = os.path.splitext(os.path.basename(file_path))[0]
    spec        = importlib.util.spec_from_file_location(module_name, file_path)
    module      = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    for name, obj in inspect.getmembers(module):
        if inspect.isclass(obj) and obj.__module__ == module.__name__: classes.append(obj)
    return classes
models = ['gemma3:1b', 'gemma3:27b'] # ['gemma3:1b', 'gemma3:4b', 'phi4', 'qwq', 'gemma3:27b', 'llama3.2', 'granite3.1-dense', 'qwen2.5:14b', 'deepseek-r1:32b']
for _filename_ in os.listdir():
    if _filename_.endswith('.py') == False: continue
    for _class_ in getClassesFromFile(_filename_):
        if issubclass(_class_, XWordsSolver):
            if _class_.__name__ not in results_lu: results_lu[_class_.__name__] = {}
            print(_class_)
            for _model_ in models:
                if _model_ in results_lu[_class_.__name__]: continue
                xwords_copy = copy.deepcopy(xwords)
                _instance_ = _class_(xwords=xwords_copy, model=_model_)
                answer_lu, request_stats, num_of_llm_requests = _instance_.solve()
                char_level_acc = xwords_copy.characterLevelAccuracy()
                results_lu[_class_.__name__][_model_] = (answer_lu, request_stats, num_of_llm_requests, char_level_acc, xwords_copy)
                print(f'\n{_class_.__name__} {_model_} {char_level_acc}')

In [ ]:
_tiles_ = []
for _algo_ in results_lu.keys():
    for _model_ in models:
        if _model_ not in results_lu[_algo_]: _xwords_ = copy.deepcopy(xwords)
        else:                                 _xwords_ = results_lu[_algo_][_model_][4]
        _tiles_.append(_xwords_.smallMultipleSVG())
rt.table(_tiles_, per_row=len(models), spacer=10)

In [ ]:
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
_xwords_ = copy.deepcopy(xwords)
_model_  = 'gemma3:27b'
def promptModel(prompt):
    class Guess(BaseModel):
        guess: str
    response: ChatResponse = chat(model=_model_, messages=[{ 'role': 'user', 'content': prompt,},],format=Guess.model_json_schema())
    guess = Guess.model_validate_json(response['message']['content'])
    return guess.guess, response['message']['content']
_prompt_ = _xwords_.createOverSpecifiedPrompt(95,'across')
print(_prompt_)
for i in range(3): print(promptModel(_prompt_))

In [ ]:
_prompt_ = '''What is the answer to the crossword puzzle clue "Steep decline" which consists of 8 letters?  The answer may be a single word or a multi-word phrase.
- The 1st letter is the 2nd letter of the clue "Not partial".
- The 2nd letter is the 2nd letter of the clue "Order after closing time".
- The 3rd letter is the 3rd letter of the clue "Author who originally intended his pen name to rhyme with 'voice,' though Americans pronounce it differenctly".
- The 4th letter is the 3rd letter of the clue "Walk unsteadily".
- The 5th letter is the 5th letter of the clue "First to the finish line".
- The 6th letter is the 1st letter of the clue "Country once led by 93-Across".
- The 7th letter is the 1st letter of the clue "Like a bodybuilder’s arms".
- The 8th letter is the 1st letter of the clue "Much-lauded Lauder".
return the answer as a JSON object.'''
promptModel(_prompt_)